In [1]:
from eyepop import EyePopSdk
from dotenv import load_dotenv
from itertools import groupby
load_dotenv()

True

In [2]:
filepath = '../assets/uploads/uploaded_image.png'
with EyePopSdk.endpoint() as endpoint:
    result = endpoint.upload(filepath).predict()

/var/folders/bx/wg__1c311y10j5x314vnv6k80000gn/T/ipykernel_96716/1430644950.py:2: DeprecationWarning: Call to deprecated function (or staticmethod) endpoint. (use workerEndpoint() instead, will be removed in v1.0.0) -- Deprecated since version 0.19.0.
  with EyePopSdk.endpoint() as endpoint:


In [89]:
res_objs = result['objects']

# find all the nutrition objects
nutritions = [
    'protein', 'fat', 'calories', 'sugar', 'sodium', 'fiber', 'carbohydrate', 'cholesterol', 'carbs', 'serving'
]
threshold = 10

# get all the objects that contain nutrition information
nutrition_objs = []
for obj in res_objs:
    # if the any of the nutrition strings are contained in the object name
    try:
        if any([nutr in obj['texts'][0]['text'].lower().replace(' ', '') for nutr in nutritions]):
            nutrition_objs.append(obj)
    except:
        print('Error:', obj)
        # remove the object from the list
        res_objs.remove(obj)
        continue
        
# Going through each nutritional information object and extracting the values and find corresponding values by finding text that are within a threshold of a 'y' value from it
extracted_nutrition = []
for obj in nutrition_objs:
    # get the 'y' value of the object
    y = obj['y']
    
    extracted_vals = []
    # loop through all the objects and find the ones that are within the threshold of the 'y' value
    for obj2 in res_objs:
        if obj2['y'] - threshold <= y <= obj2['y'] + threshold:
            val = obj2['texts'][0]['text']
            val_x = obj2['x']
            if val == obj['texts'][0]['text']:
                continue
            extracted_vals.append((val_x, val))
    
    # sort the extracted values by x value and return only the x value
    extracted_vals = [val[1] for val in sorted(extracted_vals, key=lambda x: x[0])]
    
    # add to the extracted nutrition list
    extracted_nutrition.append({
        'nutrition': obj['texts'][0]['text'],
        'values': extracted_vals
    })

Error: {'category': 'text', 'classId': 0, 'classLabel': 'text', 'confidence': 0.95, 'height': 10.462, 'id': 3411, 'orientation': 0, 'width': 42.575, 'x': 5.404, 'y': 192.683}
Error: {'category': 'text', 'classId': 0, 'classLabel': 'text', 'confidence': 0.948, 'height': 10.507, 'id': 3412, 'orientation': 0, 'width': 16.719, 'x': 258.524, 'y': 179.485}
Error: {'category': 'text', 'classId': 0, 'classLabel': 'text', 'confidence': 0.948, 'height': 8.584, 'id': 3413, 'orientation': 0, 'width': 32.022, 'x': 337.56, 'y': 66.66}
Error: {'category': 'text', 'classId': 0, 'classLabel': 'text', 'confidence': 0.947, 'height': 10.926, 'id': 3414, 'orientation': 0, 'width': 15.772, 'x': 158.704, 'y': 241.007}
Error: {'category': 'text', 'classId': 0, 'classLabel': 'text', 'confidence': 0.943, 'height': 10.695, 'id': 3415, 'orientation': 0, 'width': 25.768, 'x': 35.862, 'y': 20.564}
Error: {'category': 'text', 'classId': 0, 'classLabel': 'text', 'confidence': 0.943, 'height': 8.104, 'id': 3416, 'orie

In [90]:
obj

{'category': 'text',
 'classId': 0,
 'classLabel': 'text',
 'confidence': 0.879,
 'height': 12.023,
 'id': 3486,
 'orientation': 0,
 'texts': [{'category': 'text',
   'confidence': 0.99,
   'id': 3610,
   'text': 'Calories'}],
 'width': 49.077,
 'x': 6.732,
 'y': 48.335}

In [91]:
extracted_nutrition

[{'nutrition': 'Protein', 'values': ['3g', '5g', '3g']},
 {'nutrition': 'Cholesterol', 'values': ['15', 'mg', '35', 'D1', 'mg']},
 {'nutrition': 'Sugars',
  'values': ['32g', '32', '%', '25g', '25', '%', '16', '16']},
 {'nutrition': 'Carbohydrate', 'values': ['0g', '46', '48']},
 {'nutrition': 'Fat',
  'values': ['9g', '12', '%', '13g', '17', '%', '14', '19']},
 {'nutrition': 'Calories', 'values': ['280', '320', '330']}]

In [97]:
import re
# taking the extracted information, if values contain a-zA-Z only, then add to the nutrition name

parsed_nutrition = []
for nutr in extracted_nutrition:
    nutrition = nutr['nutrition']
    values = nutr['values']
    
    clean_vals = clean_nutrition_values(values)
    
    parsed_vals = []
    for val in clean_vals:
        if re.sub(r'[^A-Za-z0-9]', '', val).isalpha():
            if 'serving' in nutrition.lower():
                nutrition = nutrition + ' ' + val
            else:
                nutrition = val + ' ' + nutrition
        else:
            parsed_vals.append(val)
    parsed_nutrition.append({
        'nutrition': nutrition,
        'values': parsed_vals
    })
    
    
def clean_nutrition_values(values):
    clean_vals = []
    i = 0
    while i < len(values):
        # Check if the current item is a number and the next item is a unit
        if i + 1 < len(values) and values[i + 1] in units:
            clean_vals.append(values[i] + values[i + 1])  # Join number and unit
            i += 2  # Skip the unit
        else:
            clean_vals.append(values[i])  # Add the item as-is
            i += 1
            
    return clean_vals
parsed_nutrition

[{'nutrition': 'Protein', 'values': ['3g', '5g', '3g']},
 {'nutrition': 'Cholesterol', 'values': ['15mg', '35', 'D1mg']},
 {'nutrition': 'Sugars', 'values': ['32g', '32%', '25g', '25%', '16', '16']},
 {'nutrition': 'Carbohydrate', 'values': ['0g', '46', '48']},
 {'nutrition': 'Fat', 'values': ['9g', '12%', '13g', '17%', '14', '19']},
 {'nutrition': 'Calories', 'values': ['280', '320', '330']}]

In [98]:
# put into a dictionary with the values that are not percentages and if there are multiple values subscript the nnutrition name
nutrition_dict = {}
for nutr in parsed_nutrition:
    nutrition = nutr['nutrition']
    values = nutr['values']
    
    # remove all values with % in them
    values = [val for val in values if '%' not in val]
    if 'serving' in nutrition.lower():
        nutrition = 'serving'
    
    if len(values) == 1:
        nutrition_dict[nutrition.capitalize()] = values[0]
    else:
        for i, val in enumerate(values):
            nutrition_dict[f'{nutrition.capitalize()}_{i}'] = val
    
nutrition_dict

{'Protein_0': '3g',
 'Protein_1': '5g',
 'Protein_2': '3g',
 'Cholesterol_0': '15mg',
 'Cholesterol_1': '35',
 'Cholesterol_2': 'D1mg',
 'Sugars_0': '32g',
 'Sugars_1': '25g',
 'Sugars_2': '16',
 'Sugars_3': '16',
 'Carbohydrate_0': '0g',
 'Carbohydrate_1': '46',
 'Carbohydrate_2': '48',
 'Fat_0': '9g',
 'Fat_1': '13g',
 'Fat_2': '14',
 'Fat_3': '19',
 'Calories_0': '280',
 'Calories_1': '320',
 'Calories_2': '330'}

In [ ]:
{'category': 'text',
  'classId': 0,
  'classLabel': 'text',
  'confidence': 0.775,
  'height': 25.764,
  'id': 326,
  'orientation': 0,
  'texts': [{'category': 'text',
    'confidence': 0.993,
    'id': 351,
    'text': 'Calories'}],
  'width': 128.817,
  'x': 12.293,
  'y': 136.064},


{'category': 'text',
  'classId': 0,
  'classLabel': 'text',
  'confidence': 0.923,
  'height': 30.671,
  'id': 295,
  'orientation': 0,
  'texts': [{'category': 'text',
    'confidence': 0.999,
    'id': 382,
    'text': '60'}],
  'width': 52.59,
  'x': 239.148,
  'y': 130.703},